In [193]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome('C:/Users/LG/Downloads/chromedriver_win32/chromedriver')

In [194]:
from bs4 import BeautifulSoup

In [195]:
url_base = "https://brunch.co.kr/keyword/"
url_keyword= ["지구한바퀴_세계여행",
      #"감성_에세이",
      #"사랑·이별",
      #"인문학·철학",
      #"문화·예술",
      #"그림·웹툰",
      #"오늘은_이런_책",
     ]
url_end = "?q=g"

In [196]:
import requests
from tqdm import tqdm
import re

writer_cutter = re.compile('@@\w*')
scroll_num = 5
subscriber_limit = 30
post_limit = 10
post_lists = []
crawled = []

for keyword in url_keyword:
    driver.get(url_base + keyword + url_end)
    time.sleep(1)
    driver.get(url_base + keyword + url_end)
    time.sleep(1)
    for _ in tqdm(range(scroll_num)):    
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-300);")
        time.sleep(0.5)
    time.sleep(3)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    for item in soup.find_all("a",{"class":"link_post"}):
        tmp = writer_cutter.match(item.get("href")[1:])
        if not tmp == None:
            crawled.append(tmp.group())

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.92it/s]


In [202]:
for writer_id in crawled:
    driver.get("https://brunch.co.kr/" + writer_id + "#articles")
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    subscriber_count = soup.find("a",{"class":"link_count #follower"}).find("span", {"class":"num_count"}).string
    if subscriber_count[-1] == "만":
        subscriber_count = float(subscriber_count[:-1]) * 10000
    else:
        subscriber_count = float(subscriber_count)

    post_count = int(soup.find("a", {"class":"articleTab link_tab"}).find("span", {"class":"txt_tab"}).string[2:])
    if post_count >= post_limit and subscriber_count >= subscriber_limit:
        for _ in tqdm(range(scroll_num)):    
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight-300);")
            time.sleep(0.5)
        time.sleep(3)
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        for item in soup.find_all("a",{"class":"link_post"}):
            post_lists.append(item.get("href")[1:])
print(post_lists)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.93it/s]


ValueError: could not convert string to float: '2,505'

In [178]:
print(len(crawled))
print(crawled)
crawled = list(set(crawled))
print(len(crawled))
print(crawled)

54
['@@6qq0', '@@ejI', '@@1f0N', '@@ejI', '@@ejI', '@@9JzN', '@@bOkk', '@@dgOe', '@@bzNa', '@@7TX4', '@@ddf6', '@@1xRj', '@@c0MI', '@@1Mxf', '@@aOCX', '@@dbZC', '@@aiNt', '@@3fbw', '@@aSCk', '@@aA7C', '@@9Edj', '@@2eRo', '@@23TU', '@@wlQ', '@@czpj', '@@50uw', '@@aiNt', '@@ddf6', '@@23TU', '@@cJBi', '@@d1Xx', '@@cvVi', '@@3fuW', '@@1Kt0', '@@5hT6', '@@3fuW', '@@cgRg', '@@cVx8', '@@B7y', '@@McL', '@@8z7c', '@@9JzN', '@@dc8V', '@@aA7C', '@@bqul', '@@8bJs', '@@6ftO', '@@cnXy', '@@dgOe', '@@cmux', '@@uRQ', '@@bOkk', '@@JxX', '@@dfjA']
44
['@@aA7C', '@@2eRo', '@@cVx8', '@@dgOe', '@@bzNa', '@@9JzN', '@@3fuW', '@@8z7c', '@@McL', '@@6qq0', '@@cmux', '@@5hT6', '@@cnXy', '@@aSCk', '@@B7y', '@@50uw', '@@d1Xx', '@@1xRj', '@@1Kt0', '@@wlQ', '@@1f0N', '@@dc8V', '@@9Edj', '@@cJBi', '@@3fbw', '@@23TU', '@@bOkk', '@@cvVi', '@@czpj', '@@aOCX', '@@6ftO', '@@7TX4', '@@1Mxf', '@@8bJs', '@@dbZC', '@@ddf6', '@@JxX', '@@uRQ', '@@c0MI', '@@bqul', '@@dfjA', '@@ejI', '@@aiNt', '@@cgRg']


In [ ]:
for ids in crawled:
    writer_url = base_url + writer_id + "#articles"
    driver.get(writer_url)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    soup.find("a",{"class":"link_count #follower"}).find("span", {"class=""num_count"}).span.string

In [16]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

In [17]:
crawled = soup.find_all("a",{"clas":"link_post"})

In [18]:
BASE_URL = "https://brunch.co.kr/"
post_links = [BASE_URL+i["href"][1:] for i in crawled]

In [19]:
len(post_links)

220

In [20]:
import json
with open("지구한바퀴.json", "w") as f:
    json.dump(post_links,f)

In [21]:
import json
with open("./지구한바퀴.json", "rb") as f:
    post_link = json.load(f)

In [22]:
driver = webdriver.Chrome('C:/Users/LG/Downloads/chromedriver_win32/chromedriver')

In [32]:
for idx, link in enumerate(post_link):
    i = idx
    driver.get(link)
    time.sleep(1) # politeness policy
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    try:
#         print("first")
        temp = soup.find('div',{'class':"wrap_body text_align_left finish_txt"})
        text = temp.text
        if text=="":
            print("empty text")
        else:
            with open(f"./지구한바퀴/book_{i}.txt", "w", -1, 'utf-8') as f:
                f.write(text)
                print(str(i)+" success")
    except:
        try:
#             print("second")
            text = ''
            for tmp in soup.find_all('p',{'class':"wrap_item item_type_text"}):
                text += tmp.text
            if text=="":
                for tmp in soup.find_all('h4',{'class':"wrap_item item_type_text"}):
                    text += tmp.text
            with open(f"./지구한바퀴/book_{i}.txt", "w", -1, 'utf-8') as f:
                f.write(text)
                print(str(i)+" success")
        except:
            try:
#                 print("third")
                soup = BeautifulSoup(html_source, 'html.parser')
                text = ''
                for tmp in soup.find_all('h4',{'class':"wrap_item item_type_text"}):
                    text += tmp.text
                text = text.replace("\xa0", "")
                with open(f"./지구한바퀴/book_{i}.txt", "w", -1, 'utf-8') as f:
                    f.write(text)
                    print(str(i)+" success")
            except: 
                print(str(i)+" error")
                continue

0 success
1 success
2 success
3 success
4 success
5 success
6 success
7 success
8 success
9 success
10 success
11 success
12 success
13 success
14 success
15 success
16 success
17 success
18 success
19 success
20 success


KeyboardInterrupt: 

In [ ]:
text = soup.find('div',{'class':"wrap_body text_align_left finish_txt"}).text

In [ ]:
with open(f"./지구한바퀴/book_{idx}.txt", "w", -1, 'utf-8') as f:
                f.write(text)

In [ ]:
text